# Intelligent Query Processing
## Scalar UDF Inlining
Scalar User Defined Function (UDF) inlining is an example of Intelligent Query Processing that can be enabled with database compatibility level 150 or higher. Follow the steps in this notebook to see the performance difference when scalar UDF are not inlined.

**Prerequisites**: Restore the WideWorldImportersDW full backup. Then run the **extendwwidw.sql** script to customize the database. If you have already done this, proceed to the next steps

## Step 1: Create a scalar UDF
Create a function that will take a customerID as input and return a category classification based on the customer order history total spend.

In [1]:
-- Step 1: Create a new function to get a customer category based on their order spend
USE WideWorldImportersDW
GO

CREATE OR ALTER FUNCTION [Dimension].[customer_category](@CustomerKey INT) 
RETURNS CHAR(10) AS
BEGIN
DECLARE @total_amount DECIMAL(18,2);
DECLARE @category CHAR(10);

SELECT @total_amount = 	SUM([Total Including Tax]) 
FROM [Fact].[OrderHistory]
WHERE [Customer Key] = @CustomerKey

IF @total_amount < 3000000
 SET @category = 'REGULAR'
ELSE IF @total_amount < 4500000
 SET @category = 'GOLD'
ELSE 
 SET @category = 'PLATINUM'

RETURN @category
END
GO

Commands completed successfully.

Total execution time: 00:00:00.0009826

Commands completed successfully.

Total execution time: 00:00:00.0192480

## Step 2: Setup the database
Change to dbcompat 150, clear the procedure cache of the database, and warm the cache

In [2]:
-- Step 2: Set the database to db compat 150, clear the procedure cache from previous executions, and make the comparison fair by warming the cache
ALTER DATABASE WideWorldImportersDW 
SET COMPATIBILITY_LEVEL = 150
GO
ALTER DATABASE SCOPED CONFIGURATION 
CLEAR PROCEDURE_CACHE
GO
SELECT COUNT(*) FROM [Fact].[OrderHistory]
GO

Commands completed successfully.

Total execution time: 00:00:00.0042106

Commands completed successfully.

Total execution time: 00:00:00.0042789

(1 row affected)

Total execution time: 00:00:00.1079508

(No column name)
3702592


## Step 3: Run the query without scalar UDF inlining
Since the database as dbcompat = 150, use a query hint to temporarily turn off scalar UDF inlining

In [3]:
-- Step 3: Run the query but disable the use of scalar inlining using a query hint
SELECT [Customer Key], [Customer], [Dimension].[customer_category]([Customer Key]) AS [Discount Price]
FROM [Dimension].[Customer]
ORDER BY [Customer Key]
OPTION (USE HINT('DISABLE_TSQL_SCALAR_UDF_INLINING'))
GO

(403 rows affected)

Total execution time: 00:00:43.8651651

Customer Key,Customer,Discount Price
0,Unknown,PLATINUM
1,Tailspin Toys (Head Office),PLATINUM
2,"Tailspin Toys (Sylvanite, MT)",GOLD
3,"Tailspin Toys (Peeples Valley, AZ)",PLATINUM
4,"Tailspin Toys (Medicine Lodge, KS)",PLATINUM
5,"Tailspin Toys (Gasport, NY)",PLATINUM
6,"Tailspin Toys (Jessie, ND)",PLATINUM
7,"Tailspin Toys (Frankewing, TN)",PLATINUM
8,"Tailspin Toys (Bow Mar, CO)",GOLD
9,"Tailspin Toys (Netcong, NJ)",PLATINUM


## Step 4: Run the query with scalar UDF inlining
Take off the query hint and watch how faster this query runs

In [4]:
SELECT [Customer Key], [Customer], [Dimension].[customer_category]([Customer Key]) AS [Discount Price]
FROM [Dimension].[Customer]
ORDER BY [Customer Key]
GO

(403 rows affected)

Total execution time: 00:00:13.7864965

Customer Key,Customer,Discount Price
0,Unknown,PLATINUM
1,Tailspin Toys (Head Office),PLATINUM
2,"Tailspin Toys (Sylvanite, MT)",GOLD
3,"Tailspin Toys (Peeples Valley, AZ)",PLATINUM
4,"Tailspin Toys (Medicine Lodge, KS)",PLATINUM
5,"Tailspin Toys (Gasport, NY)",PLATINUM
6,"Tailspin Toys (Jessie, ND)",PLATINUM
7,"Tailspin Toys (Frankewing, TN)",PLATINUM
8,"Tailspin Toys (Bow Mar, CO)",GOLD
9,"Tailspin Toys (Netcong, NJ)",PLATINUM
